<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, d_out):
        # 순전파 시 입력을 서로 바꾸어서 곱해줌!
        dx = d_out * self.y
        dy = d_out * self.x
        return dx, dy
    
apple_box = 100
apple_box_num = 2
tax = 1.1

# 곱셈계층
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 수행
apple_box_price = mul_apple_layer.forward(apple_box, apple_box_num)
price = mul_tax_layer.forward(apple_box_price, tax)
print('순전파 수행 후 지불해야 할 최종 금액:', price)
print()

# 역전파 수행(순전파와 반대 순서로 호출)
d_price = 1
d_apple_box_price, d_tax = mul_tax_layer.backward(d_price)
d_apple_box, d_apple_box_num = mul_apple_layer.backward(d_apple_box_price)
print('역전파 수행 후 각 변수의 변화량 값')
print('사과 박스 가격:', d_apple_box)
print('사과 박스 개수:', d_apple_box_num)
print('소비자세:', d_tax)

순전파 수행 후 지불해야 할 최종 금액: 220.00000000000003

역전파 수행 후 각 변수의 변화량 값
사과 박스 가격: 2.2
사과 박스 개수: 110.00000000000001
소비자세: 200


In [2]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, d_out):
        dx = d_out * 1
        dy = d_out * 1
        return dx, dy

- 블로깅 완료
---

In [4]:
import numpy as np

In [5]:
import numpy as np

# Relu 활성함수의 역전파 계층 만들기
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x: np.array):
        self.mask = (x <= 0) # 배열 원소값이 0이하인 값들에 대한 Boolean 인덱싱
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, d_out):
        d_out[self.mask] = 0 # 순전파 시 입력값이 0이하인 데이터에 대한 국소적인 미분값은 0으로 전달하도록 변환
        dx = d_out
        
        return dx

In [6]:
# Sigmoid 활성함수의 역전파 계층 만들기 -> 결국 최종 변화량은 1(d_out) * y(1-y)가 됨!
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x: np.array):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, d_out):
        dx = d_out * self.out(1 - self.out)
        
        return dx

In [24]:
a = np.array([1,2,3])
np.expand_dims(a, axis=1)

array([[1],
       [2],
       [3]])

In [25]:
# Affine(행렬 곱) 계층 구현 -> 4차원인 경우 고려한 코드랑 차이가 있음!
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        # out = x * W + b
        self.x = x
        out = np.matmul(x, self.W) + self.b
        return out
    
    def backward(self, d_out):
        dx = np.matmul(d_out, self.W.T)
        self.dW = np.matmul(self.x.T, d_out)
        self.db = np.sum(d_out, axis=0)
        
        return dx